# Prepare final data for main approach

This notebook follows the notebook `1_prepare_data_general.ipynb` and continues with the data preparation according to Subsection 4.4.3

`1_prepare_data_general.ipynb` stopped after the first to steps described in "Restricting Sequences, Students and Classes". This is where the notebook begins.

This notebook creates the dataset `final_data_main_approach.csv` and stores it in the `data` folder. This is the data set used for the predictions.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import sys
import os
sys.path.append(os.path.abspath('../../sources'))

import utils

C:\Users\Chiara\AppData\Local\Temp\ipykernel_14024\3845850337.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\Chiara\Documents\Uni_Master_MMDS\Masterarbeit\elearning-recommender\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SAVE_PRIMARY_FILES = False

### Read Data

In [3]:
df = utils.read_data_file("data_matrix.csv")
df_orig = df.copy()
print(len(df))  # 2935101
df.head()

c:\Users\Chiara\Documents\Uni_Master_MMDS\Masterarbeit\elearning-recommender\sources\utils.py:15: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(config.DATA_FOLDER / filename)


2935101


,assignment_log_id,problem_id,timestamp,available_core_tutoring,actions,num_actions,first_answer,num_tries,final_answer,help_requested,...,hint_requested,live_tutor_requested,hint_id,explanation_id,sequence_id,student_id,class_id,teacher_id,assignment_finished,unit_test
0,12LNLV9T5P,2MZER0YXHT,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,27GEL3MV6E,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
1,XJMWRTN1F,SXFPGQ0Z7,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
2,XJMWRTN1F,C4U09P1SM,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
3,XJMWRTN1F,1Q6NCCELIN,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
4,XJMWRTN1F,36DSFWGZG,1.587659e+09,no_tutoring,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0


`sequence_relationships`

In [4]:
sequence_relationships = utils.read_sequence_relationships()
print(len(sequence_relationships))  # 12564
sequence_relationships.head()

12564


,unit_test_sequence_id,in_unit_sequence_id
0,K1U9M2PVF,1XEPEYCPC3
1,K1U9M2PVF,20SXJMMSRG
2,K1U9M2PVF,1SMS0A4N5G
3,K1U9M2PVF,1BROMSHRRA
4,K1U9M2PVF,520QV3Q8S


In [5]:
seq_test_ids = set(sequence_relationships["unit_test_sequence_id"])
seq_in_unit_ids = set(sequence_relationships["in_unit_sequence_id"])

df_seq_test_ids = set(df.loc[df["unit_test"] == 1, "sequence_id"])
df_seq_in_unit_ids = set(df.loc[df["unit_test"] == 0, "sequence_id"])

# all test ids in data_matrix are present in sequence_relationships
print(len(seq_test_ids), len(df_seq_test_ids), len(seq_test_ids & df_seq_test_ids))  # 240 151 151
# all in unit ids in data_matrix are present in sequence_relationships
print(len(seq_in_unit_ids), len(df_seq_in_unit_ids), len(seq_in_unit_ids & df_seq_in_unit_ids))  # 9186 4121 4121
# in unit ids and test ids don't overlap
print(seq_test_ids & df_seq_in_unit_ids, seq_in_unit_ids & df_seq_test_ids)  # set() set()

240 151 151
9186 4121 4121
set() set()


`assingment_relationships`

In [6]:
assignment_relationships = utils.read_assignment_relationships()
assignment_relationships = assignment_relationships.rename(
    columns={
        "unit_test_assignment_log_id": "ut_id",
        "in_unit_assignment_log_id": "iu_id",
    }
)
print(len(assignment_relationships))  # 699839
assignment_relationships.head()

699839


,ut_id,iu_id
0,7FGC8P0F1,V6YXT3UG
1,15KQFID5U5,1TFFYMT814
2,QKDRPCXSG,1N2IFGUASM
3,1JOJIQXU1B,15W4ET3W62
4,2C9YZRVZT0,1WORTY787C


In [7]:
ass_test_ids = set(assignment_relationships["ut_id"])
ass_in_unit_ids = set(assignment_relationships["iu_id"])

df_ass_test_ids = set(df.loc[df["unit_test"] == 1, "assignment_log_id"])
df_ass_in_unit_ids = set(df.loc[df["unit_test"] == 0, "assignment_log_id"])

# all test ids in data_matrix are present in assignment_relationships
print(len(ass_test_ids), len(df_ass_test_ids), len(ass_test_ids & df_ass_test_ids))  # 56577 32052 32052
# all in unit ids in data_matrix are present in assignment_relationships
print(len(ass_in_unit_ids), len(df_ass_in_unit_ids), len(ass_in_unit_ids & df_ass_in_unit_ids))
# 638528 393912 393912
# in unit ids and test ids don't overlap
print(ass_test_ids & df_ass_in_unit_ids, ass_in_unit_ids & df_ass_test_ids)  # set() set()

56577 32052 32052
638528 393912 393912
set() set()


### Restricting Sequences, Students and Classes

- Drop UT sequences that have only been completed by one class.
- Only keep assignments for which there are corresponding IU or UT assignments.
- Drop classes with less than 3 students.
- Remove classes that only completed UT assignments.
- Again, drop UT sequences that have only been completed by one class.
- Drop IU assignments from non-relevant classes that do not correspond to any of the remaining UT assignments.

#### Drop UT sequences that have only been completed by one class.


We will not analyze UT sequences which are only completed by one class as for those there cannot be determined a reference class. We will drop those first.

In [8]:
ut_seq_complete = set(df.loc[df["unit_test"] == 1, "sequence_id"])
classes_complete = set(df["class_id"])
len(ut_seq_complete), len(classes_complete)  # 151, 2071

(151, 2071)

We will keep all data for the relevant students (student which are in relevant classes) as it might be possible that there are further prediction goals for which we need more than the corresponding UT and IU assignments.

In [9]:
# drop unit test sequences on which only one class worked
num_classes_per_seq = df.loc[df["unit_test"] == 1].groupby("sequence_id")["class_id"].nunique()
drop_seq = num_classes_per_seq[num_classes_per_seq == 1].index
print(f"Drop {len(drop_seq)} test sequences as only one class worked on it")  # 31
old_len = len(df)
df_rest = df.loc[~df["sequence_id"].isin(drop_seq)].copy()
print(f"This corresponds to {old_len - len(df_rest)} records in df")  # 2744

Drop 31 test sequences as only one class worked on it
This corresponds to 2744 records in df


#### Only keep assignments for which there are corresponding IU or UT assignments.

In `assignment_relationships` it is stated, which IU assignment corresponds to which UT assignment. When applying the prediction models, we will predict the performance for the UT assignments based on the IU assignments. Therefore, we need the information from `assignment_relationships` on which IU assignments should be used.  
Then, we will drop all IU assignments that do not belong to any of the remaining UT assignments, as well as all UT assignments that do not have any IU assignments left for prediction.

To do so, we proceed as follows:
- We restrict `assignment_relationships` to those UT assignments and IU assignments that are in `df`.
- Then, we restrict `df` to those UT assignments and IU assignments that are left in `assignment_relationships`.

In [10]:
# restrict assignment_relationships to ut assignments and iu assignments in df
ut_ass = df_rest.loc[df_rest["unit_test"] == 1, "assignment_log_id"].unique()
iu_ass = df_rest.loc[df_rest["unit_test"] == 0, "assignment_log_id"].unique()
print("Number of assignments in df")
print(f"Unit test assignments: {len(ut_ass)}, in-unit assignments: {len(iu_ass)}")
# 31739, 393912

assignment_relationships = assignment_relationships.loc[
    (assignment_relationships["ut_id"].isin(ut_ass))
    & (assignment_relationships["iu_id"].isin(iu_ass))
].reset_index(drop=True)
print("assignment_relationships")
print(f"Number of rows: {len(assignment_relationships)}")  # 359801
print(f"Number of distinct test assignments: {assignment_relationships['ut_id'].nunique()}")
# 31491
print(f"Number of distinct in unit assignments: {assignment_relationships['iu_id'].nunique()}")
# 356585

Number of assignments in df
Unit test assignments: 31739, in-unit assignments: 393912
assignment_relationships
Number of rows: 359801
Number of distinct test assignments: 31491
Number of distinct in unit assignments: 356585


In [11]:
# restrict df to relevant unit test and in unit assignments
ut_ass = set(assignment_relationships["ut_id"])
iu_ass = set(assignment_relationships["iu_id"])
old_len = len(df_rest)
df_rest = df_rest.loc[
    df_rest["assignment_log_id"].isin(list(ut_ass | iu_ass))
].reset_index(drop=True)
print(f"Number of rows dropped: {old_len - len(df_rest)}")  # 226769

Number of rows dropped: 226769


#### Drop classes with less than 3 students and that only completed UT assignments.

Classes that only completed UT or IU assignments, respectively, will not be analyzed individually. In addition, we will only perform the predictions for classes with at least 3 students. However, we cannot drop the other classes completely as we may need this data for students that are in multiple classes and completed coherent assignments in those.

However, we determine which classes can be analyzed individually and which cannot. For those that cannot, we drop the students that only appear in the respective class.

In [12]:
# determine classes with at least 3 students
num_stud_per_class = df_rest.groupby("class_id")["student_id"].nunique()
rel_classes = num_stud_per_class[num_stud_per_class >= 3].index
print(f"{len(num_stud_per_class[num_stud_per_class < 3])} have less than 3 students.")  # 364
print(f"{len(rel_classes)} classes remain")  # 1626

364 have less than 3 students.
1626 classes remain


In [13]:
# determine classes that completed both sequence types
num_seq_types = (
    df_rest.loc[df_rest["class_id"].isin(rel_classes)]
    .groupby("class_id")["unit_test"]
    .nunique()
)
rel_classes = num_seq_types[num_seq_types == 2].index
num_drop = len(num_seq_types) - len(rel_classes)
print(
    f"{num_drop} (out of {len(num_seq_types)} remaining) classes only completed ut or iu sequences"
)  # 93, 1626
print(f"{len(rel_classes)} classes remain")  # 1533

93 (out of 1626 remaining) classes only completed ut or iu sequences
1533 classes remain


In [14]:
# only keep students from relevant classes
rel_students = df.loc[df["class_id"].isin(rel_classes), "student_id"].unique()
num_all_students = df["student_id"].nunique()
print(
    f"{num_all_students - len(rel_students)} out of {num_all_students} students will be dropped"
)  # 490, 21738
print(f"{len(rel_students)} students remain")  # 21248

# restrict to relevant students
old_len = len(df)
df = df.loc[df["student_id"].isin(rel_students)].copy()
print(
    f"{old_len - len(df)} rows have been deleted, resulting in {len(df)} rows"
)  # 41210, 2893891
print(
    f"{df['class_id'].nunique()} classes remain from which {len(rel_classes)} are relevant"
)  # 1786, 1533

490 out of 21738 students will be dropped
21248 students remain
41210 rows have been deleted, resulting in 2893891 rows
1786 classes remain from which 1533 are relevant


In [15]:
# drop unit test data for non-relevant classes
old_len = len(df)
df = df.loc[(df["unit_test"] == 0) | (df["class_id"].isin(rel_classes))].copy()
print(f"Drop {old_len - len(df)} rows, resulting in {len(df)} rows")  # 1100, 2892791

Drop 1100 rows, resulting in 2892791 rows


There are still UT assignments in df for which no IU assignments exist and vice versa. Those will be dropped for our predictions.

Until now, we determined the relevant classes for which predictions will be performed and our df contains all relevant data for them.

#### Again, drop UT sequences that have only been completed by one class and drop IU assignments from non-relevant classes that do not correspond to any of the remaining UT assignments.

Remove UT sequences on which only one class worked as for those there will not be found a reference class.

Remove UT assignments which do not have any corresponding IU assignments and remove IU assignments which do not belong to any of the existing UT assignments.

Moreover, remove UT assignments from non-relevant classes. We only consider the relevant classes and corresponding UT assignments. If a student completed an IU assignment in another class, this is only considered if it belongs to an UT assignment completed in a relevant classes.

In [16]:
old_num_class = df["class_id"].nunique()
old_num_stud = df["student_id"].nunique()
old_num_ass = df["assignment_log_id"].nunique()
old_len = len(df)

# drop unit test sequences on which only one class worked
num_classes_per_seq = (
    df.loc[df["unit_test"] == 1].groupby("sequence_id")["class_id"].nunique()
)
drop_seq = num_classes_per_seq[num_classes_per_seq == 1].index
print(f"Dropped {len(drop_seq)} test sequences as only one class worked on it")  # 18
old_len = len(df)
df = df.loc[~df["sequence_id"].isin(drop_seq)].copy()

# restrict assignment_relationships to ut assignments and iu assignments in df
ut_ass = df.loc[df["unit_test"] == 1, "assignment_log_id"].unique()
iu_ass = df.loc[df["unit_test"] == 0, "assignment_log_id"].unique()
assignment_relationships = assignment_relationships.loc[
    (assignment_relationships["ut_id"].isin(ut_ass))
    & (assignment_relationships["iu_id"].isin(iu_ass))
].reset_index(drop=True)

# drop assignments which do not have match
ut_ass = set(assignment_relationships["ut_id"])
iu_ass = set(assignment_relationships["iu_id"])
df = df.loc[df["assignment_log_id"].isin(list(ut_ass | iu_ass))].reset_index(drop=True)

new_num_class = df["class_id"].nunique()
num_rel_classes = df.loc[df["unit_test"] == 1, "class_id"].nunique()
new_num_stud = df["student_id"].nunique()
new_num_ass = df["assignment_log_id"].nunique()
new_len = len(df)
print(
    f"{old_num_class - new_num_class} classes have been dropped, resulting in {new_num_class} classes"
)
# 102, 1670
print(f"{num_rel_classes} classes are relevant")  # 1532
print(
    f"{old_num_stud - new_num_stud} students have been dropped, resulting in {new_num_stud} students"
)
# 79, 21169
print(
    f"{old_num_ass - new_num_ass} assignments have been dropped, resulting in {new_num_ass} assignments"
)
# 37356, 382947
print(
    f"{old_len - new_len} rows have been dropped, resulting in {new_len} rows"
)  # 228218, 2664573

Dropped 18 test sequences as only one class worked on it
102 classes have been dropped, resulting in 1670 classes
1532 classes are relevant
79 students have been dropped, resulting in 21169 students
37356 assignments have been dropped, resulting in 382947 assignments
228218 rows have been dropped, resulting in 2664573 rows


In [17]:
if SAVE_PRIMARY_FILES:
    utils.save_as_csv(df, "final_data_main_approach.csv", save_idx=False)

### Data Analysis

In [18]:
df.head()

,assignment_log_id,problem_id,timestamp,available_core_tutoring,actions,num_actions,first_answer,num_tries,final_answer,help_requested,...,hint_requested,live_tutor_requested,hint_id,explanation_id,sequence_id,student_id,class_id,teacher_id,assignment_finished,unit_test
0,12LNLV9T5P,2MZER0YXHT,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,27GEL3MV6E,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
1,XJMWRTN1F,SXFPGQ0Z7,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
2,XJMWRTN1F,C4U09P1SM,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
3,XJMWRTN1F,1Q6NCCELIN,1.587658e+09,answer,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0
4,XJMWRTN1F,36DSFWGZG,1.587659e+09,no_tutoring,"['problem_started', 'correct_response', 'probl...",3.0,1,1.0,1.0,0.0,...,0.0,0.0,NaN,NaN,1DHG1UNJ8Q,125OTT0E74,100VH25818,WOOQUZY13,1.0,0


In [19]:
# number of assignments (UT and IU)
df["assignment_log_id"].nunique(), df.loc[df["unit_test"] == 1, "assignment_log_id"].nunique(), df.loc[df["unit_test"] == 0, "assignment_log_id"].nunique()

(382947, 30876, 352071)

In [20]:
# number of problems per UT assignment
num_probs_per_ass_ut = df[df["unit_test"] == 1].groupby("assignment_log_id")["problem_id"].nunique()

# number of problems per IU assignment
num_probs_per_ass_iu = df[df["unit_test"] == 0].groupby("assignment_log_id")["problem_id"].nunique()

num_probs_per_ass_ut.describe(), num_probs_per_ass_iu.describe()

(count    30876.000000
 mean        11.362450
 std          6.576614
 min          3.000000
 25%          6.000000
 50%         10.000000
 75%         15.000000
 max         44.000000
 Name: problem_id, dtype: float64,
 count    352071.000000
 mean          6.571816
 std           6.603166
 min           1.000000
 25%           2.000000
 50%           4.000000
 75%           9.000000
 max          75.000000
 Name: problem_id, dtype: float64)

In [21]:
# number of relevant classes, students, and mean number of students in relevant classes
df.loc[df["unit_test"] == 1, "class_id"].nunique(), df["student_id"].nunique(), df[df["unit_test"] == 1].groupby("class_id")["student_id"].nunique().describe()

(1532,
 21169,
 count    1532.000000
 mean       14.531984
 std         9.121483
 min         1.000000
 25%         8.000000
 50%        13.000000
 75%        19.000000
 max       104.000000
 Name: student_id, dtype: float64)

In [22]:
# mean performance in ut data, mean performance in iu data
df.loc[df["unit_test"] == 1, "first_answer"].mean(), df.loc[df["unit_test"] == 0, "first_answer"].mean()

(0.5859469197068641, 0.6947923410780613)